# Laurels and Crowns: Ancient Roman Coins Classification Using Computer Vision

Valérie BLANCH

2614867B@student.gla.ac.uk

University of Glasgow

September 2022


# Link to the dataset
The .zip file contains the text data, the images and the saved weigths of the trained models described in the dissertation.

https://drive.google.com/file/d/1y-hHm1IWHguGPKzfZi6Q4_84LLcUvktv/view?usp=sharing

In [ ]:
pip install tf-explain

In [ ]:
#importing modules

import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import Model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import random 
from tf_explain.core.grad_cam import GradCAM
import cv2
from IPython.display import Image, display

In [ ]:
#extracting the zip file

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/coins2.zip', 'r')
zip_ref.extractall('tmp/')
zip_ref.close()

#setting the paths of the training, validation & test sets

base_dir = 'tmp/coins2'
train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_denarius_dir = os.path.join(train_dir, 'denarius') 
train_antoninianus_dir = os.path.join(train_dir, 'antoninianus') 

validation_denarius_dir = os.path.join(validation_dir, 'denarius')
validation_antoninianus_dir = os.path.join(validation_dir, 'antoninianus') 
 
test_denarius_dir = os.path.join(test_dir, 'denarius') 
test_antoninianus_dir = os.path.join(test_dir, 'antoninianus')

# Exploratory Analysis

In [ ]:
#loading the datasets

antoninianus = pd.read_csv('tmp/coins2/antoninianus.csv')
denarius = pd.read_csv('tmp/coins2/denarius.csv')

#printing several rows of the antoninianus dataframe & dimensions

antoninianus.head()

In [ ]:
#printing the dimensions of the antoninianus dataframe

antoninianus.shape

In [ ]:
#printing the dimensions of the denarius dataframe

denarius.shape

In [ ]:
#printing several rows of the denarius dataframe

denarius.head()

In [ ]:
#inspecting missing values in the relevant variables

print(antoninianus['Coin'].isna().sum())
print(denarius['Coin'].isna().sum())
print(antoninianus['Obverse_URL'].isna().sum())
print(denarius['Obverse_URL'].isna().sum())
print(antoninianus['Denomination'].isna().sum())
print(denarius['Denomination'].isna().sum())
print(antoninianus['Obverse_Description'].isna().sum())
print(denarius['Obverse_Description'].isna().sum())

#inspecting duplicates

print(antoninianus['Coin'].nunique())
print(denarius['Coin'].nunique())

#inspecting denominations

print(antoninianus['Denomination'].unique())
print(denarius['Denomination'].unique())

In [ ]:
#computing an average date from estimated range

antoninianus['Year'] = (antoninianus['From_Date'] + antoninianus['To_Date'])/2
denarius['Year'] = (denarius['From_Date'] + denarius['To_Date'])/2

#plotting dates for antoniniani

antoninianus.groupby('Year')['Coin'].nunique().plot()
plt.title('Number of antoniniani found per year of production')
plt.xlabel('Estimated year of production')
plt.ylabel('Number of coins found');

In [ ]:
#plotting dates for denarii

denarius.groupby('Year')['Coin'].nunique().plot()
plt.title('Number of denarii found per year of production')
plt.xlabel('Estimated year of production')
plt.ylabel('Number of coins found');

In [ ]:
#printing number of radiate portraits for antoniniani

print(antoninianus.Obverse_Description.str.contains('radiate').sum())

In [ ]:
#printing number of hairtsyles for antoniniani that are not radiate crowns

anton_divergent = antoninianus[~antoninianus['Obverse_Description'].str.contains('radiate',
                                                                    na=False)]
anton_divergent.shape                                    

In [ ]:
anton_divergent

In [ ]:
#displaying hairstyles for antoniniani that are not radiate crowns

for i in range(0,5):
  display(Image(anton_divergent.iloc[i,6]))

In [ ]:
#drawing a plot for the proportion of hairstyles for antoniniani

y = [antoninianus.Obverse_Description.str.contains('radiate').sum()+5, 0]
x = ['Radiate', 'Other']

plt.bar(x, y)
plt.title('Type of Hairstyles for Antoniniani')
plt.xlabel('Type of hairstyles')
plt.ylabel('Occurrences');

In [ ]:
#printing occurrences of hairstyles for denarii

print(denarius.Obverse_Description.str.contains('radiate').sum())
print(denarius.Obverse_Description.str.contains('laureate').sum())
print(denarius.Obverse_Description.str.contains('bare').sum())

In [ ]:
#inspecting radiate crowns in denarius dataset

radiate = denarius[denarius['Obverse_Description'].str.contains('radiate',
                                                                    na=False)]
radiate.shape      

In [ ]:
radiate

In [ ]:
#displaying radiate crowns in denarius dataset

for i in range(0,2):
  display(Image(radiate.iloc[i,6]))

In [ ]:
y = [2, 9568, 1811, 134]
x = ['Radiate', 'Laureate', 'Bareheaded', 'Other']

plt.bar(x, y)
plt.title('Type of Hairstyles for Denarii')
plt.xlabel('Type of hairstyles')
plt.ylabel('Occurrences');

In [ ]:
#computing the summary statistics of diameter and weight

print(antoninianus['Diameter'].describe())
print(antoninianus['Weight'].describe())
print(denarius['Diameter'].describe())
print(denarius['Weight'].describe())

# Image Pre-processing


In [ ]:
#displaying several coins of the two denominations

train_antoninianus_fnames = os.listdir( train_antoninianus_dir )
train_denarius_fnames = os.listdir( train_denarius_dir )

nrows = 4
ncols = 4

pic_index = 0 

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_antoninianus_pix = [os.path.join(train_antoninianus_dir, fname) 
                for fname in train_antoninianus_fnames[ pic_index-8:pic_index] 
               ]

next_denarius_pix = [os.path.join(train_denarius_dir, fname) 
                for fname in train_denarius_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_antoninianus_pix+next_denarius_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') 

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
#averaging the images from the train set - antoninianius

from PIL import Image

antoninianus_pix = [os.path.join(train_antoninianus_dir, fname) 
                for fname in train_antoninianus_fnames 
               ]

img=[]

for i in antoninianus_pix:

  image = load_img(i)
  image = img_to_array(image)
  img.append(image)

img = np.array(img)
avg = np.average(img,axis=0)
train_anton = Image.fromarray(avg.astype('uint8'))
plt.imshow(train_anton)
plt.axis('Off') 
plt.title('Average image from training set - antoninianus');

In [ ]:
#averaging images from the train set - denarius

denarius_pix = [os.path.join(train_denarius_dir, fname) 
                for fname in train_denarius_fnames 
               ]

img=[]

for i in denarius_pix:

  image = load_img(i)
  image = img_to_array(image)
  img.append(image)

img = np.array(img)
avg = np.average(img,axis=0)
train_den = Image.fromarray(avg.astype('uint8'))
plt.imshow(train_den)
plt.axis('Off') 
plt.title('Average image from training set - denarius');

# Model 1 :  CNN with grayscale images

In [ ]:
#preprocessing the data : rescaling, augmentation with rotation & mirroring

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   horizontal_flip = True)

val_datagen = ImageDataGenerator(rescale = 1.0/255.)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
#generating batches of 100 for training & validation sets in grayscale

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 100,
                                                    class_mode = 'categorical',
                                                    color_mode='grayscale',
                                                    target_size = (150, 150),
                                                    classes={'denarius': 0, 
                                                          'antoninianus': 1})     

validation_generator = val_datagen.flow_from_directory(validation_dir,
                                                          batch_size = 100,
                                                    class_mode = 'categorical',
                                                        color_mode='grayscale',
                                                       classes={'denarius': 0, 
                                                          'antoninianus': 1},
                                                      target_size = (150, 150))

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        class_mode='categorical',
        color_mode='grayscale',
        classes={'denarius': 0, 
        'antoninianus': 1},
        shuffle=False)

In [ ]:
#verifying the indices of the categories

print(train_generator.class_indices)

In [ ]:
#defining the layers of the model

model_1 = tf.keras.models.Sequential([                                                      
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(2, activation='softmax')
])

#printing the summary of the model

model_1.summary()

In [ ]:
#compiling the model

model_1.compile(optimizer = 'rmsprop', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
#defining a checkpoint to save the best model during training

checkpoint_path = 'tmp/model_1_new.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True, 
                                                verbose=1)

In [ ]:
%%time

#training the model

history_1 = model_1.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 140,
            epochs = 50,
            validation_steps = 70,
            callbacks=[checkpoint])

In [ ]:
#plotting the accuracy for training & validation sets

plt.plot(history_1.history['accuracy'])
plt.plot(history_1.history['val_accuracy'])
plt.title('Accuracy - Model 1')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

#plotting the loss for training & validation sets

plt.plot(history_1.history['loss'])
plt.plot(history_1.history['val_loss'])
plt.title('Loss - Model 1')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#updating the model with best weights

model_1.load_weights(checkpoint_path)
model_1.evaluate(validation_generator)

#or loading the weights used in the dissertation

#model_1.load_weights('tmp/coins2/model_1.ckpt')

In [ ]:
#making predictions
probabilities = model_1.predict(test_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test_generator.classes, y_pred, cmap='Blues')
plt.title('Confusion Matrix - Model 1')
plt.show()

In [ ]:
#printing accuracy on the test set

model_1.evaluate(test_generator)

# Model 2 : Updated CNN with grayscale images

In [ ]:
#defining the layers of the model

model_2 = tf.keras.models.Sequential([                                                      
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(2, activation='softmax')
])

#printing the summary of the model

model_2.summary()

In [ ]:
#compiling the model

model_2.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
#creating a checkpoint callback to save the best model

checkpoint_path = 'tmp/model_2_new.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True, 
                                                verbose=1)

In [ ]:
%%time

#training the model

history_2 = model_2.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 140,
            epochs = 50,
            validation_steps = 70,
            callbacks=[checkpoint])

In [ ]:
#plotting the accuracy for training & validation sets

plt.plot(history_2.history['accuracy'])
plt.plot(history_2.history['val_accuracy'])
plt.title('Accuracy - Model 2')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

#plotting the loss for training & validation sets

plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.title('Loss - Model 2')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#updating the model with best weights

model_2.load_weights(checkpoint_path)
model_2.evaluate(validation_generator)

#or loading the weights used for the dissertation

#model_2.load_weights('tmp/coins2/model_2.ckpt')

In [ ]:
#making predictions
probabilities = model_2.predict(test_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test_generator.classes, y_pred, cmap='Blues')
plt.title('Confusion Matrix - Model 2')
plt.show()

In [ ]:
#printing accuracy on the test set

model_2.evaluate(test_generator)

In [ ]:
#creating a dataframe with predictions and filenames

pred_df =  pd.DataFrame(y_pred, test_generator.filenames).reset_index()
pred_df = pred_df.rename(columns={'index': 'filenames', 0: 'predictions'})
pred_df['classes'] = test_generator.classes

#subsetting mislabelled antoniniani

anton_df = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 1)]

#subsetting mislabelled denarii

den_df = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 0)]

#subsetting true antoniniani

anton_df_true = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 1)]

#subsetting true denarii

den_df_true = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 0)]

In [ ]:
#displaying mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  sp = plt.subplot(len(den_df), ncols,i+1)
  image = load_img('tmp/coins2/test/' + den_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  sp = plt.subplot(len(anton_df), ncols,i+1)
  image = load_img('tmp/coins2/test/' + anton_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying gradCAM of mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  path = 'tmp/coins2/test/' + den_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150), 
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=1, layer_name='conv2d_6')
  plt.subplot(len(den_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);


In [ ]:
#displaying gradCAM of mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  path = 'tmp/coins2/test/' + anton_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150), 
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=0, layer_name='conv2d_6')
  plt.subplot(len(anton_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true antoniniani (first layer)

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test/' + anton_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150),
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=1, layer_name='conv2d_3')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true antoniniani (second layer)

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test/' + anton_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150),
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=1, layer_name='conv2d_4')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true antoniniani (third layer)

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test/' + anton_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150),
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=1, layer_name='conv2d_5')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true antoniniani (fourth layer)

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test/' + anton_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150),
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=1, layer_name='conv2d_6')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true denarii

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test/' + den_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150),
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=0, layer_name='conv2d_6')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

# Model 3 : CNN with RGB images

In [ ]:
#new generator objects with RGB images

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 100,
                                                    target_size = (150, 150),
                                                   class_mode='categorical',
                                                    classes={'denarius': 0, 
                                                          'antoninianus': 1})     

validation_generator = val_datagen.flow_from_directory(validation_dir,
                                                          batch_size = 100,
                                                      target_size = (150, 150),
                                                       class_mode='categorical',
                                                       classes={'denarius': 0, 
                                                          'antoninianus': 1})


test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        shuffle=False,
       class_mode='categorical',
        classes={'denarius': 0, 
                'antoninianus': 1})

In [ ]:
#creating the cnn

model_3 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])

#printing Print the model summary
model_3.summary()

In [ ]:
#compiling the model

model_3.compile(optimizer = 'rmsprop', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
#creating a checkpoint callback to save the best model

checkpoint_path = 'tmp/model_3_new.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True, 
                                                verbose=1)

In [ ]:
%%time

#training the model

history_3 = model_3.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 140,
            epochs = 50,
            validation_steps = 70,
            callbacks=[checkpoint])

In [ ]:
#plotting the accuracy for training & validation sets

plt.plot(history_3.history['accuracy'])
plt.plot(history_3.history['val_accuracy'])
plt.title('Accuracy - Model 3')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

#plotting the loss for training & validation sets

plt.plot(history_3.history['loss'])
plt.plot(history_3.history['val_loss'])
plt.title('Loss - Model 3')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#updating the model with best weights

model_3.load_weights(checkpoint_path)
model_3.evaluate(validation_generator)

#or loading the weights used for the dissertation

#model_3.load_weights('tmp/coins2/model_3.ckpt')

In [ ]:
#making predictions

probabilities = model_3.predict(test_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test_generator.classes, y_pred, cmap='Blues')
plt.title('Confusion Matrix - Model 3')

print(test_generator.class_indices)

In [ ]:
#printing accuracy on the test set

model_3.evaluate(test_generator)

# Model 4 : Updated CNN with RGB images

In [ ]:
#creating the cnn

model_4 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

#printing Print the model summary
model_4.summary()

In [ ]:
#compiling the model

model_4.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
#creating a checkpoint callback to save the best model

checkpoint_path = 'tmp/model_4_new.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True, 
                                                verbose=1)

In [ ]:
%%time

#training the model

history_4 = model_4.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 140,
            epochs = 50,
            validation_steps = 70,
            callbacks=[checkpoint])

In [ ]:
#plotting the accuracy for training & validation sets

plt.plot(history_4.history['accuracy'])
plt.plot(history_4.history['val_accuracy'])
plt.title('Accuracy - Model 4')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

#plotting the loss for training & validation sets

plt.plot(history_4.history['loss'])
plt.plot(history_4.history['val_loss'])
plt.title('Loss - Model 4')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#updating the model with best weights

model_4.load_weights(checkpoint_path)
model_4.evaluate(validation_generator)

#or loading the weights used for the dissertation

#model_4.load_weights('tmp/coins2/model_4.ckpt')

In [ ]:
#making predictions

probabilities = model_4.predict(test_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test_generator.classes, y_pred, cmap='Blues')
plt.title('Confusion Matrix - Model 4')

print(test_generator.class_indices)

In [ ]:
#printing accuracy on the test set

model_4.evaluate(test_generator)

In [ ]:
#creating a dataframe with predictions and filenames

pred_df =  pd.DataFrame(y_pred, test_generator.filenames).reset_index()
pred_df = pred_df.rename(columns={'index': 'filenames', 0: 'predictions'})
pred_df['classes'] = test_generator.classes

#subsetting mislabelled antoniniani

anton_df = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 1)]

#subsetting mislabelled denarii

den_df = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 0)]

#subsetting true antoniniani

anton_df_true = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 1)]

#subsetting true denarii

den_df_true = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 0)]

In [ ]:
#displaying mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  sp = plt.subplot(len(den_df), ncols,i+1)
  image = load_img('tmp/coins2/test/' + den_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  sp = plt.subplot(len(anton_df), ncols,i+1)
  image = load_img('tmp/coins2/test/' + anton_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying gradCAM of mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  path = 'tmp/coins2/test/' + den_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_4, class_index=1, layer_name='conv2d_16')
  plt.subplot(len(den_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);


In [ ]:
#displaying gradCAM of mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  path = 'tmp/coins2/test/' + anton_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_4, class_index=0, layer_name='conv2d_16')
  plt.subplot(len(anton_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true antoniniani

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test/' + anton_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150, 3))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_4, class_index=1, layer_name='conv2d_16')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true denarii

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test/' + den_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_4, class_index=0, layer_name='conv2d_16')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

# Model 5 : Transfert Learning with Inception V3

In [ ]:
#downloading inceptionv3 without the final layers

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

#saving it into a variable

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

#setting the input shape

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

#loading the weights

pre_trained_model.load_weights(local_weights_file)

#freezing the layers of the model

for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
#printing the layers of the pretrained model

pre_trained_model.summary()

In [ ]:
#only using the model up to the 7th module

last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
#adding layers to the pretrained model
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(2, activation='softmax')(x)           

model_5 = Model(pre_trained_model.input, x) 

In [ ]:
#compiling the model

model_5.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

#printing the model summary 

model_5.summary()

In [ ]:
#creating a checkpoint callback to save the best model

checkpoint_path = 'tmp/model_5_new.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True, 
                                                verbose=1)

In [ ]:
%%time

#training the model

history_5 = model_5.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 140,
            epochs = 50,
            validation_steps = 70,
            callbacks=[checkpoint])

In [ ]:
#plotting the accuracy for training & validation sets

plt.plot(history_5.history['accuracy'])
plt.plot(history_5.history['val_accuracy'])
plt.title('Accuracy - Model 5')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

#plotting the loss for training & validation sets

plt.plot(history_5.history['loss'])
plt.plot(history_5.history['val_loss'])
plt.title('Loss - Model 5')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#updating the model with best weights

model_5.load_weights(checkpoint_path)
model_5.evaluate(validation_generator)

#or loading weights used for dissertation

#model_5.load_weights('tmp/coins2/model_5.ckpt')

In [ ]:
#making predictions

probabilities = model_5.predict(test_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test_generator.classes, y_pred, cmap='Blues')
plt.title('Confusion Matrix - Model 5')
plt.show()

print(test_generator.class_indices)

In [ ]:
#printing accuracy on the test set

model_5.evaluate(test_generator)

# Model 6 : Updated InceptionV3

In [ ]:
#only using the model up to the 4th module

last_layer = pre_trained_model.get_layer('mixed4')
last_output = last_layer.output

In [ ]:
#adding layers to the pre-trained model

x = layers.Conv2D(128, (3,3), activation='relu')(last_output)
x = tf.keras.layers.MaxPooling2D(2,2)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x) 
x = layers.Dense(2, activation='softmax')(x)           

#building the model

model_6 = Model(pre_trained_model.input, x) 

In [ ]:
#compiling the model

model_6.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

#printing the summary of the model

model_6.summary()

In [ ]:
#creating a checkpoint callback to save the best model

checkpoint_path = 'tmp/model_6_new.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True, 
                                                verbose=1)

In [ ]:
%%time

#training the model

history_6 = model_6.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 140,
            epochs = 50,
            validation_steps = 70,
            callbacks=[checkpoint])

In [ ]:
#plotting the accuracy for training & validation sets

plt.plot(history_6.history['accuracy'])
plt.plot(history_6.history['val_accuracy'])
plt.title('Accuracy - Model 6')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

#plotting the loss for training & validation sets

plt.plot(history_6.history['loss'])
plt.plot(history_6.history['val_loss'])
plt.title('Loss - Model 6')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#updating the model with best weights

model_6.load_weights(checkpoint_path)
model_6.evaluate(validation_generator)

#or loading weights used for the dissertation

#model_6.load_weights('tmp/coins2/model_6.ckpt')

In [ ]:
#making predictions

probabilities = model_6.predict(test_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test_generator.classes, y_pred, cmap='Blues')
plt.title('Confusion Matrix - Model 6')
plt.show()

print(test_generator.class_indices)

In [ ]:
#printing accuracy on the test set

model_6.evaluate(test_generator)

In [ ]:
#creating a dataframe with predictions and filenames

pred_df =  pd.DataFrame(y_pred, test_generator.filenames).reset_index()
pred_df = pred_df.rename(columns={'index': 'filenames', 0: 'predictions'})
pred_df['classes'] = test_generator.classes

#subsetting mislabelled antoniniani

anton_df = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 1)]

#subsetting mislabelled denarii

den_df = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 0)]

#subsetting true antoniniani

anton_df_true = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 1)]

#subsetting true denarii

den_df_true = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 0)]

In [ ]:
#displaying mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  sp = plt.subplot(len(anton_df), ncols,i+1)
  image = load_img('tmp/coins2/test/' + anton_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  sp = plt.subplot(len(den_df), ncols,i+1)
  image = load_img('tmp/coins2/test/' + den_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying gradCAM of mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  path = 'tmp/coins2/test/' + anton_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_6, class_index=0, layer_name='conv2d_111')
  plt.subplot(len(anton_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  path = 'tmp/coins2/test/' + den_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_6, class_index=1, layer_name='conv2d_111')
  plt.subplot(len(den_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true antoniniani

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test/' + anton_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_6, class_index=1, layer_name='conv2d_111')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true denarii

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test/' + den_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_6, class_index=0, layer_name='conv2d_111')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#testing the two denarius with radiate crowns

path = 'tmp/coins2/validation/denarius/den_9480.jpg'
img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150,3))
img =tf.keras.preprocessing.image.img_to_array(img)
img /= 255
img = ([img], None)

explainer = GradCAM()
grid = explainer.explain(
        img, model_6, class_index=0, layer_name='conv2d_111')
grid = Image.fromarray(grid)
plt.axis('Off')
plt.imshow(grid);

In [ ]:
#testing the two denarius with radiate crowns

path = 'tmp/coins2/validation/denarius/den_11240.jpg'
img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150,3))
img =tf.keras.preprocessing.image.img_to_array(img)
img /= 255
img = ([img], None)

explainer = GradCAM()
grid = explainer.explain(
        img, model_6, class_index=0, layer_name='conv2d_111')
grid = Image.fromarray(grid)
plt.axis('Off')
plt.imshow(grid);

# New Test Set

In [ ]:
#drawing a stacked bar plot of the distribution of collections per denomination

anton_coll = pd.DataFrame(antoninianus['Collection_URI'].value_counts()).reset_index()
den_coll = pd.DataFrame(denarius['Collection_URI'].value_counts()).reset_index()

#renaming columns

anton_coll = anton_coll.rename(columns={'index': 'Collection_URI', 'Collection_URI': 'antoninianus'})
den_coll = den_coll.rename(columns={'index': 'Collection_URI', 'Collection_URI': 'denarius'})

#merging dataframes

collections = pd.merge(den_coll, anton_coll, how='left')

#transposing the dataframe

collections = pd.DataFrame.transpose(collections)
collections = (collections.rename(columns=collections.iloc[0])).drop(collections.index[0])

#drawing the plot

plot = collections.plot.bar(stacked=True, rot=0)
plot.set_title('Distribution of coins per type and collection')
plot.legend(loc='center left',bbox_to_anchor=(1.0, 0.5));

In [ ]:
#loading the new test set

test2_dir = os.path.join(base_dir, 'test2')

test2_denarius_dir = os.path.join(test2_dir, 'denarius') 
test2_antoninianus_dir = os.path.join(test2_dir, 'antoninianus')

In [ ]:
#displaying several coins

test2_antoninianus_fnames = os.listdir(test2_antoninianus_dir)
test2_denarius_fnames = os.listdir(test2_denarius_dir)

nrows = 4
ncols = 4

pic_index = 0 

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_antoninianus_pix = [os.path.join(test2_antoninianus_dir, fname) 
                for fname in test2_antoninianus_fnames[ pic_index-8:pic_index] 
               ]

next_denarius_pix = [os.path.join(test2_denarius_dir, fname) 
                for fname in test2_denarius_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_antoninianus_pix+next_denarius_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') 

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
#averaging the images from the second test set - antoninianius

antoninianus_pix = [os.path.join(test2_antoninianus_dir, fname) 
                for fname in test2_antoninianus_fnames 
               ]

img=[]

for i in antoninianus_pix:

  image = load_img(i)
  image = img_to_array(image)
  img.append(image)

img = np.array(img)
avg = np.average(img,axis=0)
test2_anton = Image.fromarray(avg.astype('uint8'))
plt.imshow(test2_anton)
plt.axis('Off') 
plt.title('Average image from second test set - antoninianus');

In [ ]:
#averaging images from the second test set - denarius

denarius_pix = [os.path.join(test2_denarius_dir, fname) 
                for fname in test2_denarius_fnames 
               ]

img=[]

for i in denarius_pix:

  image = load_img(i)
  image = img_to_array(image)
  img.append(image)

img = np.array(img)
avg = np.average(img,axis=0)
test2_den = Image.fromarray(avg.astype('uint8'))
plt.imshow(test2_den)
plt.axis('Off') 
plt.title('Average image from second test set - denarius');

## Model 1 - Second Test

In [ ]:
#preprocessing the data

test2_datagen = ImageDataGenerator(rescale = 1.0/255.)

test2_generator = test2_datagen.flow_from_directory(
        test2_dir,
        target_size=(150, 150),
        class_mode='categorical',
        color_mode='grayscale',
        classes={'denarius': 0, 
        'antoninianus': 1},
        shuffle=False)

In [ ]:
#making predictions

probabilities = model_1.predict(test2_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test2_generator.classes, y_pred, cmap='Oranges')
plt.title('Confusion Matrix - Model 1 - Second Test')
plt.show()

print(test2_generator.class_indices)

In [ ]:
#printing accuracy of the second test

model_1.evaluate(test2_generator)

## Model 2 - Second Test

In [ ]:
#making predictions

probabilities = model_2.predict(test2_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test2_generator.classes, y_pred, cmap='Oranges')
plt.title('Confusion Matrix - Model 2 - Second Test')
plt.show()

print(test2_generator.class_indices)

In [ ]:
#printing accuracy of the second test

model_2.evaluate(test2_generator)

In [ ]:
#creating a dataframe with predictions and filenames

pred_df =  pd.DataFrame(y_pred, test2_generator.filenames).reset_index()
pred_df = pred_df.rename(columns={'index': 'filenames', 0: 'predictions'})
pred_df['classes'] = test2_generator.classes

#subsetting mislabelled antoniniani

anton_df = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 1)]

#subsetting mislabelled denarii

den_df = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 0)]

#subsetting true antoniniani

anton_df_true = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 1)]

#subsetting true denarii

den_df_true = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 0)]

In [ ]:
#displaying mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  sp = plt.subplot(len(den_df), ncols,i+1)
  image = load_img('tmp/coins2/test2/' + den_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  sp = plt.subplot(len(anton_df), ncols,i+1)
  image = load_img('tmp/coins2/test2/' + anton_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying gradCAM of mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  path = 'tmp/coins2/test2/' + den_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150), 
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=1, layer_name='conv2d_6')
  plt.subplot(len(den_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);


In [ ]:
#displaying gradCAM of mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  path = 'tmp/coins2/test2/' + anton_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150), 
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=0, layer_name='conv2d_6')
  plt.subplot(len(anton_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true antoniniani

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test2/' + anton_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150),
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=1, layer_name='conv2d_6')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true denarii

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test2/' + den_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150),
                                            color_mode='grayscale')
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_2, class_index=0, layer_name='conv2d_6')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

## Model 3 - Second Test

In [ ]:
#new generator objects with RGB images

test2_generator = test2_datagen.flow_from_directory(
        test2_dir,
        target_size=(150, 150),
        shuffle=False,
       class_mode='categorical',
        classes={'denarius': 0, 
                'antoninianus': 1})

In [ ]:
#making predictions

probabilities = model_3.predict(test2_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test2_generator.classes, y_pred, cmap='Oranges')
plt.title('Confusion Matrix - Model 3 - Second Test')
plt.show()

print(test2_generator.class_indices)

In [ ]:
#printing the accuracy of the second test

model_3.evaluate(test2_generator)

## Model 4 - Second Test

In [ ]:
#making predictions

probabilities = model_4.predict(test2_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test2_generator.classes, y_pred, cmap='Oranges')
plt.title('Confusion Matrix - Model 4 - Second Test')
plt.show()

print(test2_generator.class_indices)

In [ ]:
#printing the accuracy of the second test

model_4.evaluate(test2_generator)

In [ ]:
#creating a dataframe with predictions and filenames

pred_df =  pd.DataFrame(y_pred, test2_generator.filenames).reset_index()
pred_df = pred_df.rename(columns={'index': 'filenames', 0: 'predictions'})
pred_df['classes'] = test2_generator.classes

#subsetting mislabelled antoniniani

anton_df = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 1)]

#subsetting mislabelled denarii

den_df = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 0)]

#subsetting true antoniniani

anton_df_true = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 1)]

#subsetting true denarii

den_df_true = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 0)]

In [ ]:
#displaying mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  sp = plt.subplot(len(den_df), ncols,i+1)
  image = load_img('tmp/coins2/test2/' + den_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  sp = plt.subplot(len(anton_df), ncols,i+1)
  image = load_img('tmp/coins2/test2/' + anton_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying gradCAM of mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  path = 'tmp/coins2/test2/' + den_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_4, class_index=1, layer_name='conv2d_16')
  plt.subplot(len(den_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);


In [ ]:
#displaying gradCAM of mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  path = 'tmp/coins2/test2/' + anton_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_4, class_index=0, layer_name='conv2d_16')
  plt.subplot(len(anton_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true antoniniani

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test2/' + anton_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150, 3))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_4, class_index=1, layer_name='conv2d_16')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true denarii

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test2/' + den_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_4, class_index=0, layer_name='conv2d_16')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

## Model 5 - Second Test

In [ ]:
#making predictions

probabilities = model_5.predict(test2_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test2_generator.classes, y_pred, cmap='Oranges')
plt.title('Confusion Matrix - Model 5 - Second Test')
plt.show()

print(test2_generator.class_indices)

In [ ]:
#printing the accuracy of the second test 

model_5.evaluate(test2_generator)

## Model 6 - Second Test

In [ ]:
#making predictions

probabilities = model_6.predict(test2_generator)

#computing a confusion matrix

y_pred = np.argmax(probabilities, axis=1)

ConfusionMatrixDisplay.from_predictions(test2_generator.classes, y_pred, cmap='Oranges')
plt.title('Confusion Matrix - Model 6 - Second Test')
plt.show()

print(test2_generator.class_indices)

In [ ]:
#printing the accuracy of the second test 

model_6.evaluate(test2_generator)

In [ ]:
#creating a dataframe with predictions and filenames

pred_df =  pd.DataFrame(y_pred, test2_generator.filenames).reset_index()
pred_df = pred_df.rename(columns={'index': 'filenames', 0: 'predictions'})
pred_df['classes'] = test2_generator.classes

#subsetting mislabelled antoniniani

anton_df = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 1)]

#subsetting mislabelled denarii

den_df = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 0)]

#subsetting true antoniniani

anton_df_true = pred_df.loc[(pred_df['predictions'] == True) & (pred_df['classes'] == 1)]

#subsetting true denarii

den_df_true = pred_df.loc[(pred_df['predictions'] == False) & (pred_df['classes'] == 0)]

In [ ]:
#displaying mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  sp = plt.subplot(len(anton_df), ncols,i+1)
  image = load_img('tmp/coins2/test2/' + anton_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  sp = plt.subplot(len(den_df), ncols,i+1)
  image = load_img('tmp/coins2/test2/' + den_df.iloc[i,0])
  plt.axis('Off')
  plt.title(i)
  plt.imshow(image);

In [ ]:
#displaying gradCAM of mislabelled antoniniani

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(anton_df)*4)

for i in range(0,len(anton_df)):
  path = 'tmp/coins2/test2/' + anton_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_6, class_index=0, layer_name='conv2d_111')
  plt.subplot(len(anton_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of mislabelled denarii

ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, len(den_df)*4)

for i in range(0,len(den_df)):
  path = 'tmp/coins2/test2/' + den_df.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_6, class_index=1, layer_name='conv2d_111')
  plt.subplot(len(den_df), ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);


In [ ]:
#displaying gradCAM of true antoniniani

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test2/' + anton_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_6, class_index=1, layer_name='conv2d_111')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);

In [ ]:
#displaying gradCAM of true negatives (=denarii)

ncols = 4
nrows = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

for i in range(0,16):
  path = 'tmp/coins2/test2/' + den_df_true.iloc[i,0]
  img=tf.keras.preprocessing.image.load_img(path, target_size=(150, 150))
  img =tf.keras.preprocessing.image.img_to_array(img)
  img /= 255
  img = ([img], None)
  explainer = GradCAM()
  grid = explainer.explain(
        img, model_6, class_index=0, layer_name='conv2d_111')
  plt.subplot(nrows, ncols,i+1)
  grid = Image.fromarray(grid)
  plt.axis('Off')
  plt.title(i)
  plt.imshow(grid);